## README

This notebook demonstrates how to interact with a **Question Answering Multi-Agent System (QAMAS)** built using the **Reflex framework**. Reflex provides an event-driven environment that supports the orchestration of modular, reactive agents to collaboratively answer complex user queries.

This notebook is specifically tailored for completing the **GAIA Hands-on Challenge** from the [Hugging Face Agents Course – Unit 4](https://huggingface.co/learn/agents-course/unit4/hands-on). It automates the process of retrieving evaluation questions, generating answers using a Reflex-powered multi-agent architecture, and submitting the responses back to Hugging Face for scoring.

---

### Notebook Structure

The notebook is organized into three key stages:

1. **Extract Phase**  
   Retrieves a set of GAIA questions using the Hugging Face API. These questions are part of the official evaluation and require high-quality, reliable responses.

2. **GAIA Question Answering**  
   Uses the Reflex multi-agent system to answer each question by dynamically routing it through the appropriate agent pipeline. This phase showcases the collaborative and reactive capabilities of the architecture.

3. **Load Phase**  
   Submits the generated answers to the Hugging Face evaluation endpoint and retrieves scores that reflect the system's performance.

---

### System Architecture

The QAMAS follows a modular, pipeline-based architecture where each agent has a specialized role and communicates asynchronously via the Reflex environment. The interaction is initiated by the **Router Agent**, which delegates the query to a suitable path based on the nature of the question. Each path ends with a **Verifier Agent** ensuring the accuracy and quality of the response before the final answer is returned.

#### Agent Overview

- **Router Agent**  
  The Router analyzes each GAIA question and dynamically routes it to the correct downstream agent(s):
  - **Factual or up-to-date queries** → Researcher
  - **Logical/mathematical reasoning** → Reasoner
  - **Structured/tabular data** → Data Analyst

- **Data Analysis Agent**  
  Specializes in interpreting structured data (e.g., CSVs, tables) and performing:
  - Aggregations
  - Pattern recognition
  - Calculations and filtering
  - Format-compliant reporting

- **Researcher Agent**  
  Gathers external or current information from reliable sources using:
  - Search queries
  - Clarifying sub-questions
  - Web tools or APIs (if available)
  - Source evaluation and synthesis

- **Reasoner Agent**  
  Handles logical and mathematical queries by:
  - Applying formal reasoning techniques
  - Executing step-by-step deduction or computation
  - Validating solutions with alternative approaches when necessary

- **Generator Agent (Initial & Final)**  
  Responsible for transforming intermediate outputs into concise final answers. Ensures:
  - Clean formatting
  - Adherence to expected answer type (e.g., string, list, number)
  - Incorporation of verification feedback

- **Verifier Agent**  
  Evaluates the quality of the generated answer:
  - Confirms factual and logical accuracy
  - Ensures strict format compliance
  - Highlights inconsistencies or omissions

  If issues are found, it routes feedback to the Generator for answer refinement. This feedback loop improves both precision and robustness, especially important for evaluation benchmarks like GAIA.

---

### Agent Pipeline Graph

Depending on the nature of each GAIA question, the system dynamically selects one of the following processing routes:

- **Structured Data Questions**  
  `Router → Data Analyst → Generator → Verifier → Generator`

- **Factual + Reasoning Questions (Multi-hop)**  
  `Router → Researcher → Reasoner → Generator → Verifier → Generator`

- **Logical/Mathematical Questions**  
  `Router → Reasoner → Generator → Verifier → Generator`

Each pipeline concludes with a **Verifier-Generator** cycle that improves answer fidelity and ensures conformity to GAIA’s evaluation format and quality expectations.

<div align="center">
  <img src="images/multi_agents_graph.png" alt="Multi-Agent Pipeline Graph"/>
  <center><em>Figure 1: Multi-Agent Pipeline Graph showing different processing routes</em></center>
</div>

---

This notebook serves both as a demonstration of Reflex-based agent collaboration and as a working solution for the Hugging Face GAIA evaluation challenge.

In [ ]:
import sys
sys.path.append("../")

In [ ]:
import os
import huggingface_hub
import time

from src.agent import question_answering
from src.data import extract, load
from src.tools.startup import settings

## Parameters

In [ ]:
graph_config = {
    "configurable": {
        "thread_id": "1"}, 
    "recursion_limit": 30
}

questions_file_path = os.path.join(
    settings["volumes"]["raw"], "gaia_questions.json")

## 1. Extract Phase

Logging to Hugginface 

In [ ]:
huggingface_hub.login(os.environ["HF_TOKEN"])

In [ ]:
if not os.path.exists(questions_file_path):
    questions = extract.get_questions(settings["volumes"]["interim"])
else:
    questions = extract.read_json_file(questions_file_path)

## 2. GAIA Questions Answering

In [ ]:
answers = []
for i, question in enumerate(questions, start=1):
    print(f"Question {i}: {question['question']}")
    print("*"*30)
    time.sleep(5)

    # Execute the agents with the GAIA question
    qa_agent = question_answering.QuestionAnsweringAgent(graph_config)
    try:
        answer = qa_agent.answer_gaia_question(
            question, stream_mode="values", subgraphs=False, debug=False)
    except Exception as _:
        answer = ""

    # Save answer
    answers.append({
      "task_id": question["task_id"],
      "submitted_answer": answer
    })

## 3. Load Phase

In [ ]:
if not os.path.exists(questions_file_path):
    load.save_json_file(questions, questions_file_path)

In [ ]:
response = load.submit_answers(answers)
response